# Follow up answering RAG

In [20]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_google_genai.llms import GoogleGenerativeAI
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.output_parsers import StrOutputParser
from langsmith.run_helpers import traceable
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from operator import itemgetter
from typing import List
from langchain_core.documents import Document

import sys
sys.path.append("../")
from src.config import Configuration


class FollowUpChain:
    def __init__(self, config: Configuration, doc_content: str) -> None:
        # chat model
        self.chat_model = ChatGoogleGenerativeAI(
            model="gemini-pro",
            temperature=0,
            convert_system_message_to_human=True,
            google_api_key=config.load_gemini_token()
        )
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", f'You are a helpful admission assitant for Ton Duc Thang university.\nGive you the following context to answer my questions. Output "None" if you cannot answer:\n```\n{doc_content}\n```\n'),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{question}")])

        self.memory = ConversationBufferMemory(return_messages=True)
        self.chain = RunnablePassthrough.aschain = (
            {
                "question": itemgetter("question")
            }
            |RunnablePassthrough.assign(
                history=RunnableLambda(self.memory.load_memory_variables) | itemgetter("history"),
            )
            | self.prompt
            | self.chat_model
        )
        return

    @traceable(tags=["followup"])
    def answer(self, question: str) -> str:
        inputs = {"question": question}
        response = self.chain.invoke(inputs)
        self.memory.save_context(inputs=inputs, outputs={"output": response.content})
        return response



# Test

In [2]:
from src.rag.hybrid_gemini import HybridGeminiRag
config = Configuration()
rag = HybridGeminiRag(
    config=config, 
    es_index="labse-major", 
    embed_model="sentence-transformers/LaBSE")

From (44, 4) to Counter({2: 37, 1: 6, 3: 1})
From (44, 4) to 44


In [3]:
from pprint import pprint
rag_resp = rag.ask_rag(question="Em muốn biết thêm về ngành du lịch của trường")
rag_resp.answer


'- Sinh viên sau khi tốt nghiệp có khả năng phân tích vấn đề, giá trị, xu hướng của du lịch trong thời kỳ mới và bối cảnh kinh tế toàn cầu.\n- Sinh viên có thể phân tích, đánh giá và vận dụng kiến thức chuyên ngành của khoa học quản lý vào kinh doanh du lịch, đặc biệt trong lĩnh vực kinh doanh điểm đến du lịch và công nghệ giải trí du lịch.\n- Sinh viên có thể khai thác sản phẩm du lịch, tổ chức quản lý trong hoạt động du lịch.'

In [21]:
config.enable_tracing(project="LEARN")
followup = FollowUpChain(config=Configuration(), doc_content=rag_resp.document.content)

In [22]:
q = "Ngành học này tên là gì?"
a = followup.answer(question=q)
a

AIMessage(content='Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch')

In [6]:
followup.memory.load_memory_variables({})

{'history': [HumanMessage(content='Ngành học này tên là gì?'),
  AIMessage(content='Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch')]}

In [7]:
q = "Vậy cử nhân ngành này có thể ứng tuyển vào vị trí nào?"
a = followup.answer(question=q)
a

AIMessage(content='Cử nhân Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch có thể ứng tuyển vào các vị trí như:\n- Chuyên viên cung cấp dịch vụ du lịch tại các khu du lịch, điểm vui chơi tham quan du lịch\n- Chuyên gia tư vấn về chiến lược kinh doanh, khai thác, xây dựng và cung cấp sản phẩm du lịch\n- Chuyên viên thuộc Bộ và Sở văn hóa – Thể thao – Du lịch như Trung tâm bồi dưỡng nghiệp vụ du lịch, Viện nghiên cứu và phát triển du lịch, Quy hoạch du lịch, Trung tâm xúc tiến du lịch\n- Tự tạo lập doanh nghiệp du lịch mới với quy mô nhỏ hoặc tham gia giảng dạy tại các cơ sở đào tạo về du lịch')

In [23]:
a.content

'Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch'

In [8]:
followup.memory.load_memory_variables({})

{'history': [HumanMessage(content='Ngành học này tên là gì?'),
  AIMessage(content='Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch'),
  HumanMessage(content='Vậy cử nhân ngành này có thể ứng tuyển vào vị trí nào?'),
  AIMessage(content='Cử nhân Việt Nam học – Chuyên ngành Du lịch và Quản lý du lịch có thể ứng tuyển vào các vị trí như:\n- Chuyên viên cung cấp dịch vụ du lịch tại các khu du lịch, điểm vui chơi tham quan du lịch\n- Chuyên gia tư vấn về chiến lược kinh doanh, khai thác, xây dựng và cung cấp sản phẩm du lịch\n- Chuyên viên thuộc Bộ và Sở văn hóa – Thể thao – Du lịch như Trung tâm bồi dưỡng nghiệp vụ du lịch, Viện nghiên cứu và phát triển du lịch, Quy hoạch du lịch, Trung tâm xúc tiến du lịch\n- Tự tạo lập doanh nghiệp du lịch mới với quy mô nhỏ hoặc tham gia giảng dạy tại các cơ sở đào tạo về du lịch')]}